<h2> Files</h2>
<body>
    <b>smiles </b>: Virtual Environment - Use source /smiles/bin/activate to activate the virtual env<br>
    <b>datasets/download.sh </b>: Bash script to download datasets 
		Usage : bash download.sh gdb13 OR bash download.sh gdb17 or bash download.sh gdb17
<br>
    <b>datasets/delete.sh </b>: bash script to delete files in the folder. 
		Usage : bash delete.sh gdb11_ where gdb11 is the grep string to search for.
<br>
    <b>datasets/extract.sh </b>: To extract files from a tar file and to save it in a folder

<br>

 

# Functions

In [ ]:
max(1,2)

2

In [ ]:
# Function to One hot encode the data into into 2 Dimensions. 
# Splitting the dataset to len(distinct_charachter) number of columns
def OneHotEncoding(data,ohe_dict,max):
  """ The function makes use of the ohe_helper function to One Hot Encode a list of charachters and to return 
  the encoded format. Since the input here is a string, the output will be of 2 Dimensions """

  import numpy as np
  ohe=[]
  for data_point in data:
    ohe.append(ohe_helper(data_point,ohe_dict,max))
  return ohe

def ohe_helper(data,ohe_dict,max_value):
  """ Helper function to One Hot Encode the data. 
  It is used by the above function"""

  import numpy as np
  maximum=max(max_value,len(ohe_dict))
  out=np.zeros((maximum,maximum))
  for i,element in enumerate(data):
    out[i][key_value[element]]=1
  return out

def save(file,key_values):
  """ The function will take the data and save it as an npz file, with
  name smiles.npz"""
  import numpy as np
  np.savez('smiles.npz',ohe=temp,key=key_values)

def OneHotDecoding_helper(data,dictionary):
  """ Function to decode a One Hot Encoded 2D data back into its smile representation"""
  key=list(dictionary.keys())
  string=''
  for charachter in data:
    idx=charachter.argmax()
    string+=key[idx]
  return string

def check_smiles(string):
  object=check_molecule(string)

In [ ]:
from IPython.core.display import Image
class gan:
  def __init__(self):
    """ The function will check if there is a pretrained model already in the 
    folder. If found, it will load the data. Otherwise, it will return error message"""

    print('Class initialized')
    import os
    from keras.models import  load_model
    flag=0
    for file in os.listdir():
      if file=='smiles_generator_dcgan.h5':
        self.model=load_model(file)
        print('Pre trained model loaded')
        flag=1
    if flag==0:
      print('Pre trained model not found')
            
  

  def create_generator(self,image_size=28,input_size=100):
    ''' The function creates a generative model use=ing Keras and returns the model
    The image is first converted into a smaller dimension with more channels. 
    Once this is done, we use futher Conv2D transpose operation to increase the size of the imahe
    and to decrese the number of channels.
    After the process, the model ends up creating an image of size 28x28 in the last layer. 
    '''

    import keras 
    import tensorflow as tf
    #Build an input layer
    gen_input = tf.keras.layers.Input(shape=(input_size,))
    
    #Increase dimensions and resize to 3D to feed it to Conv2DTranspose layer
    x = tf.keras.layers.Dense(7 * 7 * 128)(gen_input)
    x = tf.keras.layers.Reshape((7, 7, 128))(x)
    
    #Use ConvTranspose
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2DTranspose(128, kernel_size=[5,5], strides=2, padding='same')(x)
    
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2DTranspose(64, kernel_size=[5,5], strides=2, padding='same')(x)
    
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2DTranspose(32, kernel_size=[5,5], strides=1, padding='same')(x)
    
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2DTranspose(1, kernel_size=[5,5], strides=1, padding='same')(x)
    
    #Output layer for Generator
    x = tf.keras.layers.Activation('sigmoid')(x)
    
    #Build model using Model API
    generator = tf.keras.models.Model(gen_input, x, name='generator')
    
    return generator

    
  def create_discriminator(self,shape=[28,28,1,]):
      """ Function to build the discriminator network from scratch using the tensorflow 
      and keras libraries"""
      import tensorflow as tf
      #Build the network
      dis_input = tf.keras.layers.Input(shape)
      x = tf.keras.layers.LeakyReLU(alpha=0.2)(dis_input)
      x = tf.keras.layers.Conv2D(32, kernel_size=[5,5], strides=2, padding='same')(x)
      
      x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
      x = tf.keras.layers.Conv2D(64, kernel_size=[5,5], strides=2, padding='same')(x)
      
      x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
      x = tf.keras.layers.Conv2D(128, kernel_size=[5,5], strides=2, padding='same')(x)
      
      x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
      x = tf.keras.layers.Conv2D(256, kernel_size=[5,5], strides=1, padding='same')(x)
      
      #Flatten the output and build an output layer
      x = tf.keras.layers.Flatten()(x)
      x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
      
      #Build Model
      discriminator = tf.keras.models.Model(dis_input, x, name='discriminator')
      
      return discriminator



  def build_model(self):
    """ The function creates a GAN model by using the discrimnator and generator models which were
    defined in the earlier funtions inside the class. It also returns the generator, disciminator and 
    adversarial network after creating the model"""
    import tensorflow as tf

    noise_size = 100
    lr = 2e-4
    decay = 6e-8
    
    #Build Base Discriminator model
    base_discriminator = self.create_discriminator()
    
    #Define optimizer and compile model
    discriminator = tf.keras.models.Model(inputs=base_discriminator.inputs, 
                                          outputs=base_discriminator.outputs)
    optimizer = tf.keras.optimizers.RMSprop(lr=lr, decay=decay)
    discriminator.compile(loss='binary_crossentropy',
                          optimizer=optimizer,
                          metrics=['accuracy'])
    
    #Build Generator model
    generator = self.create_generator(image_size=28, input_size=noise_size)
    
    #Build Frozen Discriminator
    frozen_discriminator = tf.keras.models.Model(inputs=base_discriminator.inputs, 
                                          outputs=base_discriminator.outputs)
    #Freeze the weights of discriminator during adversarial training
    frozen_discriminator.trainable = False

    #Build Adversarial model
    optimizer = tf.keras.optimizers.RMSprop(lr=lr * 0.5, decay=decay * 0.5)
    #Adversarial = generator + discriminator
    adversarial = tf.keras.models.Model(generator.input, 
                        frozen_discriminator(generator.output))
    
    adversarial.compile(loss='binary_crossentropy',
                        optimizer=optimizer,
                        metrics=['accuracy'])    
    
    return generator, discriminator, adversarial





  def train_gan(self,generator,discriminator,adversarial_network,key_value,distribution_size=100):
    import tensorflow as tf
    import numpy as np
    #Training parameters
    batch_size = 64
    train_steps = 30000
    image_size = 28
    

    train_x=data_en
    #Make it 3D dataset
    train_x = np.reshape(train_x, [-1, image_size, image_size, 1])
    #Standardize data : 0 to 1
    train_x = train_x.astype('float32') / 255
    
    #Input for testing generator at different intervals, we will generate 16 images
    test_noise_input = np.random.uniform(-1.0,1.0, size=[16, distribution_size])
    
    #Start training
    for i in range(train_steps):
        
        #Train DISCRIMATOR
        
        #1. Get fake images from Generator
        noise_input = np.random.uniform(-1.0,1.0, size=[batch_size, distribution_size])
        fake_images = generator.predict(noise_input)
        
        #2. Get real images from training set
        img_indexes = np.random.randint(0, train_x.shape[0], size=batch_size)
        real_images = train_x[img_indexes]
        
        #3. Prepare input for training Discriminator
        X = np.concatenate((real_images, fake_images))
        
        #4. Labels for training
        y_real = np.ones((batch_size, 1))
        y_fake = np.zeros((batch_size, 1))
        y = np.concatenate((y_real, y_fake))
        
        #5. Train Discriminator
        d_loss, d_acc = discriminator.train_on_batch(X, y)
        
        
        #Train ADVERSARIAL Network
        
        #1. Prepare input - create a new batch of noise
        X = noise_input = np.random.uniform(-1.0,1.0, size=[batch_size, distribution_size])
        
        #2. Prepare labels - training Adversarial network to lie :) - All 1s
        y = np.ones((batch_size, 1))
        
        #3. Train - Pls note Discrimator is not getting trained here
        a_loss, a_acc = adversarial_network.train_on_batch(X, y)
        
        if i % 100 == 0:
            #Print loss and Accuracy for both networks
            print("%s [Discriminator loss: %f, acc: %f, Adversarial loss: %f, acc: %f]" % (i, d_loss, d_acc, a_loss, a_acc) )
        
        #Save generated images to see how well Generator is doing
        if (i+1) % 500 == 0:
            
            #Generate 16 images
            fake_images = generator.predict(test_noise_input)
            
            #Display images
            self.plot_images(fake_images, i+1)

    #Save Generator model
    self.model=generator
    generator.save('smiles_generator_dcgan.h5')   




  def plot_images(self,fake_images, step):
    '''  The function helps in plotting the graphs, so that it is easy to 
    see how the model trains '''
    
    import matplotlib.pyplot as plt
    import math
    import numpy as np
    plt.figure(figsize=(2.5,2.5))
    num_images = fake_images.shape[0]
    
    image_size = fake_images.shape[1]
    rows = int(math.sqrt(fake_images.shape[0]))
    
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(fake_images[i], [image_size, image_size])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.show()

  def generate_smiles(self,number):
    ''' This function will generate 'number' number of 5's and will return the array'''
    import numpy as np
    noise_input = np.random.uniform(-1.0,1.0, size=[number, 100])
    fake_images = self.model.predict(noise_input)
    return fake_images


# Preparing the data

In [ ]:
# Connecting to google drive and navigating to folder
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("drive/MyDrive/Colab Notebooks/molecule_generation_SMILES/scripts")

In [ ]:
!pip install smilite
# Importing Libraries
import pandas as pd
import smilite # Library to check if data generated is correct or not
from check_molecule import *

In [ ]:
% time
data=pd.read_csv('gdb11_size10.smi',delimiter='\t',header=None)
# Renaming the columns
data.columns=['Molecule','_','__']

# Dropping the extra columns
data.drop(data.columns[-2:],axis=1,inplace=True)

# Fetching a smaller portion of the dataset
data=data.iloc[0:500000]

data.head()

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 5.48 µs


,Molecule
0,CC(C(C)(C)C)C(C)(C)C
1,CC(C)(C)C(N)C(C)(C)C
2,CC(C)(C)C(O)C(C)(C)C
3,CC(C)(C)C(F)C(C)(C)C
4,CC(C(C)(C)C)C(C)(C)N


<h4>One Hot Encoding the data

In [ ]:
% time

# Finding the distinct charachters in the dataset
distinct_charachters=set(' '.join(data['Molecule'].values))

# Creating a dictionary to get the index value
key_value={}
key_value[' ']=0
idx=1
for charachter in distinct_charachters:
  if charachter!=' ':
    key_value[charachter]=idx
    idx+=1

# Creating a new column with the charachter seperated as a list
data['Molecule_sep']=data['Molecule'].apply(lambda x: list(x))

#Creating a new column with length of each smiles representation
data['length']=data['Molecule_sep'].apply(lambda x: len(x))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [ ]:

#Since the charachters are not ordinal , converting them into one Hot Encoding
import numpy as np
temp=np.array(OneHotEncoding(data['Molecule_sep'],key_value,data['length'].max()))
save(temp,np.array(key_value))
% time

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 6.91 µs


In [ ]:
# Reading the saved Nump file
import numpy as np
data_en=np.load('smiles.npz',allow_pickle=True)
key_value=data_en['key'].item()
data_en=data_en['ohe']

# GAN 

In [ ]:
gan_object=gan()
Generator,Discriminator,Adversarial=gan_object.build_model()


Class initialized
Pre trained model loaded


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
gan_object.train_gan(Generator,Discriminator,Adversarial,key_value=key_value)

0 [Discriminator loss: 0.696752, acc: 0.242188, Adversarial loss: 0.814808, acc: 0.000000]
100 [Discriminator loss: 0.000006, acc: 1.000000, Adversarial loss: 0.053412, acc: 1.000000]
200 [Discriminator loss: 0.000000, acc: 1.000000, Adversarial loss: 0.000317, acc: 1.000000]
300 [Discriminator loss: 0.000211, acc: 1.000000, Adversarial loss: 0.190849, acc: 1.000000]
400 [Discriminator loss: 0.763169, acc: 0.500000, Adversarial loss: 0.196474, acc: 1.000000]


IndexError: ignored

In [ ]:
temp=gan_object.generate_smiles(5)